# 🧠 GraphRAG: Load and Query Markdown Files

This notebook prepares the markdown files (from the /markdown folder) to be used in a GraphRAG workflow.

It uses LangChain + OpenAI or HuggingFace embeddings to build a vector index + knowledge graph.


In [1]:
!pip install langchain ollama
!pip install faiss-cpu networkx tiktoken
!pip install -U langchain langchain-community
!pip install sentence-transformers


In [2]:
from pathlib import Path
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings

import os

In [3]:
markdown_folder = Path("markdown")
documents = []

for file in markdown_folder.glob("*.md"):
    loader = TextLoader(str(file), encoding='utf-8')
    documents.extend(loader.load())

print(f"Loaded {len(documents)} documents from markdown/")

Loaded 19 documents from markdown/


## ✂️ Split Text into Chunks

After loading all the markdown files as documents, we need to break them down into smaller, manageable pieces before creating embeddings.

This is important because:
- Large language models have token limits (e.g., 4,000 or 8,000 tokens per call)
- Smaller chunks allow for more accurate and efficient retrieval
- Overlapping chunks preserve context across boundaries

We’ll use `RecursiveCharacterTextSplitter` from LangChain to:
- Split each document into chunks of 500 characters
- Maintain a 100-character overlap between chunks to retain continuity

You can adjust `chunk_size` and `chunk_overlap` depending on your needs or the model’s context window.


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
docs = text_splitter.split_documents(documents)

## 🧱 Create Vectorstore with Local Embeddings

In this step, we convert the chunked text into vector representations (embeddings) so that they can be efficiently searched using semantic similarity.

Although we’re using **LLaMA 3 via Ollama** for answering questions, we still need a dedicated embedding model to build the vector index. In this setup, we use **`sentence-transformers/all-MiniLM-L6-v2`** from HuggingFace, which runs fully offline and is well-optimized for semantic similarity.

This hybrid approach ensures:
- Embeddings are fast and local
- No API keys or external services are required
- LLaMA 3 focuses on generating natural language answers


Make sure you have `sentence-transformers` installed:
```bash
pip install sentence-transformers
```

Then we create a FAISS index using those embeddings.


In [5]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(docs, embedding)

/var/folders/ny/tk1xvwsn7n11pkl0vlgl1sz80000gn/T/ipykernel_76502/2170713855.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
# --- Step 6: Create retriever and QA chain ---
retriever = vectordb.as_retriever()
llm = Ollama(model="llama3")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

/var/folders/ny/tk1xvwsn7n11pkl0vlgl1sz80000gn/T/ipykernel_76502/1390154710.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [7]:
# --- Step 7: Ask a question! ---
query = "What are the main methods used in neural decoding for speech synthesis?"
result = qa_chain({"query": query})

print("\nAnswer:", result['result'])
print("\nSources:")
for doc in result['source_documents']:
    print("-", doc.metadata['source'])


/var/folders/ny/tk1xvwsn7n11pkl0vlgl1sz80000gn/T/ipykernel_76502/1288188338.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



Answer: According to the provided context, previous strategies for neural decoding of speech production focused on:

1. Reconstructing spectrotemporal auditory representations
2. Direct classification of speech segments like phonemes or words

These approaches have limitations in scaling to larger vocabulary sizes and communication rates.

It's also mentioned that decoding of auditory cortex responses has been more successful.

Sources:
- markdown/Anumanchipalli et al. - 2019 - Speech synthesis from neural decoding of spoken sentences.md
- markdown/Bellier et al. - 2023 - Music can be reconstructed from human auditory cortex activity using nonlinear decoding models.md
- markdown/Murad and Rahimi - 2025 - Unveiling Thoughts A Review of Advancements in EEG Brain Signal Decoding into Text.md
- markdown/Chen et al. - 2024 - A neural speech decoding framework leveraging deep learning and speech synthesis.md
